# Working in Jupyter Notebook with IBM Cloud

Python support is provided through a fork of the boto3 library with features to make the most of IBM Cloud Object Storage.

It can be installed from the Python Package Index through pip install ibm-cos-sdk.

In [1]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError
import pandas as pd
import io

In [53]:
cos_credentials={
  "apikey": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "xxxxxxxxxxxxxxxxxxxxxxxx",
  "iam_apikey_name": "xxxxxxxxxxxxxxxxxxxxxx",
  "iam_role_crn": "xxxxxxxxxxxxxxxxxxxxxxxxx",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx:"
}
# Constants for IBM COS values
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token' # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
service_endpoint = 'https://s3.private.eu-de.cloud-object-storage.appdomain.cloud'

# Client operations
A client provides a low-level interface to the COS S3 API. This allows for processing HTTP responses directly, rather than making use of abstracted methods and attributes provided by a resource to access the information contained in headers or XML response payloads.

In [106]:
#Create client
cos = ibm_boto3.client('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                        ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [107]:
dir(cos)

['_PY_TO_OP_NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_client_config',
 '_convert_to_request_dict',
 '_emit_api_params',
 '_endpoint',
 '_exceptions',
 '_exceptions_factory',
 '_get_waiter_config',
 '_load_exceptions',
 '_loader',
 '_make_api_call',
 '_make_request',
 '_register_handlers',
 '_request_signer',
 '_response_parser',
 '_serializer',
 '_service_model',
 'abort_multipart_upload',
 'add_legal_hold',
 'can_paginate',
 'complete_multipart_upload',
 'copy',
 'copy_object',
 'create_bucket',
 'create_multipart_upload',
 'delete_bucket',
 'delete_bucket_analytics_configuration',
 'delete_bucket_cors',
 'delete_bucket_invent

## List current buckets 


In [ ]:
cos.list_buckets()

## List Buckets

In [76]:
for bucket in cos.list_buckets()['Buckets']:
    print(bucket['Name'])

apachesparkfundamentals-donotdelete-pr-krj4dovovltwtz
jupyternotebookproject-donotdelete-pr-ijhhmanwzaepps
mypersonalbucket1
ruslanmv-bucket


## Creating sample dataframe

In [27]:
# Import pandas library
import pandas as pd

# initialize list of lists
data = [['tom', 10], ['nick', 15], ['juli', 14]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age'])

# print dataframe.
df


,Name,Age
0,tom,10
1,nick,15
2,juli,14


In [10]:
df.to_csv('out.csv') 

In [12]:
!pwd

/home/wsuser/work


In [11]:
!ls

out.csv


In [13]:
#importing the os module
import os

#to get the current working directory
directory = os.getcwd()

In [18]:

# Join various path components
file_name=os.path.join(directory, "out.csv")

In [19]:
print(file_name)

/home/wsuser/work/out.csv


## File Uploads

In [28]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = cos
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [29]:
upload_file(file_name,'ruslanmv-bucket')

True

In [23]:
# Upload file out.csv from /home/wsuser/work/ folder into project bucket as out.csv
cos.upload_file(Filename=file_name,Bucket='ruslanmv-bucket',Key='out.csv')


## File Downloads


In [35]:
cos.download_file(Bucket='ruslanmv-bucket',Key='out.csv',Filename='downloaded_out.csv')

In [36]:
!ls

downloaded_out.csv  out2.csv  out.csv


In [51]:
# download file like object 
with open('object_out.csv', 'wb') as data:
    cos.download_fileobj('ruslanmv-bucket', 'out.csv', data)

In [52]:
!ls -ltr

total 12
-rw-rw---- 1 wsuser wscommon 39 Feb 26 15:24 out.csv
-rw-rw---- 1 wsuser wscommon 39 Feb 26 15:24 downloaded_out.csv
-rw-rw---- 1 wsuser wscommon 39 Feb 26 15:37 object_out.csv


from ibm_botocore.client import Config
import ibm_boto3

def download_file_cos(local_file_name,bucket,key):  
    s3_client = cos
    try:
        res=s3_client.download_file(Bucket=bucket,Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [68]:
download_file_cos('downloaded_out2.csv','ruslanmv-bucket','out.csv')

File Downloaded


## Create/Delete Buckets

In [ ]:
cos.create_bucket(Bucket='ruslanmv-bucket1-test')

In [79]:
for bucket in cos.list_buckets()['Buckets']:
    print(bucket['Name'])

apachesparkfundamentals-donotdelete-pr-krj4dovovltwtz
jupyternotebookproject-donotdelete-pr-ijhhmanwzaepps
mypersonalbucket1
ruslanmv-bucket
ruslanmv-bucket1-test


## Create/Delete Buckets

In [ ]:
cos.delete_bucket(Bucket='ruslanmv-bucket1-test')

In [110]:
for bucket in cos.list_buckets()['Buckets']:
    print(bucket['Name'])

apachesparkfundamentals-donotdelete-pr-krj4dovovltwtz
jupyternotebookproject-donotdelete-pr-ijhhmanwzaepps
mypersonalbucket1
ruslanmv-bucket


For more commands you can visit the documentation of IBM S3 [here]( https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-python) 

**Congratulations!** we have practice how to upload and download files by using s3 in IBM Cloud